In [1]:
import keras
import pickle

model = keras.models.load_model('models/my_model_custom')

with open('utils/classes_custom.pkl','rb') as file:
  classes = pickle.load(file)

with open('utils/tokenizer_custom.pkl','rb') as file:
  tokenizer = pickle.load(file)

with open('utils/label_encoder_custom.pkl','rb') as file:
  label_encoder = pickle.load(file)

In [2]:
import numpy as np
import pandas as pd
from nltk import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from keras_preprocessing.sequence import pad_sequences

In [13]:
class IntentClassifier:
    def __init__(self,classes,model,tokenizer,label_encoder):
        self.classes = classes
        self.classifier = model
        self.tokenizer = tokenizer
        self.label_encoder = label_encoder
        
    def remove_stop(self, strings, stop_list):
        classed = [s for s in strings if s not in stop_list]
        return classed
    
    def normalize(self, text):
        return " ".join(text)
        
    def clean(self, text):
        #convert_to_dataframe
        self.data = pd.DataFrame({'text': text})
        #convert_to_lower_case
        self.data["lower"] = self.data.text.map(lambda x : x.lower())
        #word_tokenize
        self.data["tokenized"] = self.data.lower.map(word_tokenize)
        #stopwords_remove
        stop = stopwords.words("english")
        stop_punc = list(set(punctuation)) + stop
        self.data["selected"] = self.data.tokenized.map(lambda df: self.remove_stop(df, stop_punc))
        #normalize
        lemmatizer = WordNetLemmatizer()
        self.data["lemmatized"] = self.data.selected.map(lambda xs: [lemmatizer.lemmatize(x) for x in xs])
        self.data["normalized"] = self.data.lemmatized.apply(self.normalize)
        return self.data.normalized

    def sigmoid(self, scores):
        self.scores = np.negative(scores)
        self.exp = np.exp(self.scores)
        self.scores = 1 / (1 + self.exp)
        return self.scores
    
    def get_intent(self,text):
        self.text = self.clean([text])
        self.test_keras = self.tokenizer.texts_to_sequences(self.text)
        self.test_keras_sequence = pad_sequences(self.test_keras, maxlen=12, padding='post')
        self.pred = self.classifier.predict(self.test_keras_sequence)
        #print(self.sigmoid(self.pred))
        return self.label_encoder.inverse_transform(self.pred)[0][0]
    #prediction_test = y_encoder.inverse_transform(model.predict(test_padded))

In [15]:
nlu = IntentClassifier(classes,model,tokenizer,label_encoder)
print(nlu.get_intent("the wifi is not showing"))

1/1 [==============================] - 0s 18ms/step
Hardware Issue
